# "Bibliophile Analytics: Exploring Post-2000 Publications, User Reviews, and Author Ratings in a Book Database"

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

I've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

I will perform the following tasks, in order to proceed to further analysis:

- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.

Import the libraries:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### create a database connection:

In [2]:

db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Study the tables

In [3]:
# Load the dataframes from the database
books = pd.io.sql.read_sql("SELECT * FROM books LIMIT 5;", con=engine)
authors = pd.io.sql.read_sql("SELECT * FROM authors LIMIT 5;", con=engine)
publishers = pd.io.sql.read_sql("SELECT * FROM publishers LIMIT 5;", con=engine)
ratings = pd.io.sql.read_sql("SELECT * FROM ratings LIMIT 5;", con=engine)
reviews = pd.io.sql.read_sql("SELECT * FROM reviews LIMIT 5;", con=engine)



In [4]:
# Display the sample of 5 rows from each dataframe
print("Sample of 5 rows from books:")
display(books.head())

print("\nSample of 5 rows from authors:")
display(authors.head())

print("\nSample of 5 rows from publishers:")
display(publishers.head())

print("\nSample of 5 rows from ratings:")
display(ratings.head())

print("\nSample of 5 rows from reviews:")
display(reviews.head())


Sample of 5 rows from books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Sample of 5 rows from authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Sample of 5 rows from publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Sample of 5 rows from ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Sample of 5 rows from reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Description of the data

**books**

Contains data on books:

- `book_id`
- `author_id`
- `title`
- `num_pages` — number of pages
- `publication_date`
- `publisher_id`

**authors**

Contains data on authors:

- `author_id`
- `author`

**publishers**

Contains data on publishers:

- `publisher_id`
- `publisher`

**ratings**

Contains data on user ratings:

- `rating_id`
- `book_id`
- `username` — the name of the user who rated the book
- `rating`

**reviews**

Contains data on customer reviews:

- `review_id`
- `book_id`
- `username` — the name of the user who reviewed the book
    
    
- `text` — the text of the review


#### checking for missing values and duplicates:

In [5]:
query = """
    SELECT *
    FROM books
    WHERE book_id IS NULL
        OR author_id IS NULL
        OR title IS NULL
        OR num_pages IS NULL
        OR publication_date IS NULL
        OR publisher_id IS NULL
        OR book_id IN (
            SELECT book_id
            FROM books
            GROUP BY book_id
            HAVING COUNT(*) > 1
        );
    """

info_books = pd.read_sql_query(query, con=engine)
print(info_books)


Empty DataFrame
Columns: [book_id, author_id, title, num_pages, publication_date, publisher_id]
Index: []


In [6]:
query="""SELECT *
FROM authors
WHERE author_id IS NULL OR author IS NULL OR author_id IN (
    SELECT author_id
    FROM authors
    GROUP BY author_id
    HAVING COUNT(*) > 1
);
 """
info_authors = pd.read_sql_query(query, con=engine)
print(info_authors)

Empty DataFrame
Columns: [author_id, author]
Index: []


In [7]:
query="""SELECT *
FROM publishers
WHERE publisher_id IS NULL
   OR publisher IS NULL
   OR publisher_id IN (
       SELECT publisher_id
       FROM publishers
       GROUP BY publisher_id
       HAVING COUNT(*) > 1
   );
"""

info_publishers = pd.read_sql_query(query, con=engine)
print(info_publishers)

Empty DataFrame
Columns: [publisher_id, publisher]
Index: []


In [8]:
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query="""SELECT *
FROM ratings
WHERE rating_id IS NULL
   OR book_id IS NULL
   OR username IS NULL
   OR rating IS NULL
   OR rating_id IN (
       SELECT rating_id
       FROM ratings
       GROUP BY rating_id
       HAVING COUNT(*) > 1
   );
"""

info_ratings = pd.read_sql_query(query, con=engine)
print(info_ratings)

Empty DataFrame
Columns: [rating_id, book_id, username, rating]
Index: []


In [10]:
query="""SELECT *
FROM reviews
WHERE review_id IS NULL
   OR book_id IS NULL
   OR username IS NULL
   OR text IS NULL
   OR review_id IN (
       SELECT review_id
       FROM reviews
       GROUP BY review_id
       HAVING COUNT(*) > 1
   );
"""

info_reviews = pd.read_sql_query(query, con=engine)
print(info_reviews)

Empty DataFrame
Columns: [review_id, book_id, username, text]
Index: []


The result "Empty DataFrame" means that the query returned a table with no rows. In this case, it means that there are no missing values or duplicates in the "reviews" dataframe.


***There are no missing values or duplicates in the datraframes.***

### Find the number of books released after January 1, 2000.

In [11]:
query = "SELECT COUNT(*) AS num_books FROM books WHERE publication_date > '2000-01-01';"

# Execute the SQL query and store the results in a pandas DataFrame
df_1 = pd.read_sql(query, con=engine)

# Print the result
print(df_1)

   num_books
0        819


In [12]:
print(f'Number of books released after January 1, 2000: {df_1["num_books"][0]}')

Number of books released after January 1, 2000: 819


After the first of January 2000, were released 819 books

### Find the number of user reviews and the average rating for each book.

In [13]:
reviews_count = pd.read_sql_query("""
    SELECT b.title AS book_name,
COUNT(DISTINCT r.review_id) AS reviews_count,
AVG(ra.rating) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings ra ON b.book_id = ra.book_id
GROUP BY b.book_id
ORDER BY reviews_count DESC;
""", engine)

display(reviews_count)



,book_name,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


In [14]:
reviews_count_by_title = pd.read_sql_query("""
    SELECT b.title AS book_name,
COUNT(DISTINCT r.review_id) AS reviews_count,
AVG(ra.rating) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings ra ON b.book_id = ra.book_id
GROUP BY b.title
ORDER BY reviews_count DESC;
""", engine)

display(reviews_count_by_title)


,book_name,reviews_count,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


In [15]:
ratings_count = pd.read_sql_query("""SELECT book_id, COUNT(*) AS num_ratings
FROM ratings
GROUP BY book_id
ORDER BY num_ratings DESC;
""", engine)

display(ratings_count)


,book_id,num_ratings
0,948,160
1,750,88
2,673,86
3,75,84
4,302,82
...,...,...
995,465,2
996,55,2
997,652,2
998,790,2


we have a lot more ratings than reviews, probably because it's faster just rate a book than writing a whole review.

### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help to exclude brochures and similar publications from your analysis).

In [16]:
query = """
SELECT publishers.publisher, COUNT(*) AS num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 1;
"""

# Execute the SQL query and store the results in a pandas DataFrame
df_3 = pd.read_sql(query, con=engine)

# Print the result
print(f'The publisher that has released the greatest number of books with more than 50 pages is {df_3["publisher"][0]} with {df_3["num_books"][0]} books.')


The publisher that has released the greatest number of books with more than 50 pages is Penguin Books with 42 books.


In [17]:
b=pd.read_sql_query("""
SELECT publishers.publisher, COUNT(*) AS num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 10;
""", engine)

print(b)

                   publisher  num_books
0              Penguin Books         42
1                    Vintage         31
2   Grand Central Publishing         25
3           Penguin Classics         24
4           Ballantine Books         19
5                     Bantam         19
6                    Berkley         17
7         St. Martin's Press         14
8              Berkley Books         14
9  William Morrow Paperbacks         13


The publisher that has released the greatest number of books with more than 50 pages is Penguin Books with 42 books.

### Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [18]:
c=pd.read_sql_query("""
SELECT authors.author, AVG(ratings.rating) AS avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(*) >= 50
)
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 10;
""", engine)

print(c)

                                              author  avg_rating
0                         J.K. Rowling/Mary GrandPré    4.287097
1                  Markus Zusak/Cao Xuân Việt Khương    4.264151
2                                     J.R.R. Tolkien    4.246914
3                                  Louisa May Alcott    4.192308
4                                       Rick Riordan    4.080645
5                                    William Golding    3.901408
6                                      J.D. Salinger    3.825581
7           Paulo Coelho/Alan R. Clarke/Özdemir İnce    3.789474
8  William Shakespeare/Paul Werstine/Barbara A. M...    3.787879
9                                         Lois Lowry    3.750000


In [19]:
query = """
SELECT authors.author, AVG(ratings.rating) AS avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(*) >= 50
)
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1;
"""

df_4 = pd.read_sql(query, con=engine)

# Print the result
print(f'The author with the highest average book rating is {df_4["author"][0]} with {df_4["avg_rating"][0]} score.')


The author with the highest average book rating is J.K. Rowling/Mary GrandPré with 4.287096774193548 score.


Considering books with at least 50 ratings, the author with the highest average book rating is J.K. Rowling, with the illustrator Mary GrandPré with an average score rating of 4.287096774193548.

### Find the average number of text reviews among users who rated more than 50 books. 

In [20]:
query = """
SELECT AVG(num_reviews) as avg_num_reviews
FROM (
    SELECT COUNT(*) as num_reviews
    FROM reviews
    WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(*) > 50
    )
    GROUP BY username
) as subquery;
"""

df_5 = pd.read_sql(query, con=engine)

#Print the result

print(f'The average number of text reviews among users who rated more than 50 books is {df_5["avg_num_reviews"][0]}')

The average number of text reviews among users who rated more than 50 books is 24.333333333333332


In [21]:
display(df_5)

,avg_num_reviews
0,24.333333


### Conclusions:

+ After the first of January 2000, were released 819 books
+ Users tend to rate books easily, but not to write a full review, therefore for each book there are much more ratings than reviews. we see a maximum of 8 reviews for a book
+ The publisher that has released the greatest number of books with more than 50 pages is Penguin Books with 42 books.
+ Considering books with at least 50 ratings, the author with the highest average book rating is J.K. Rowling, with the illustrator Mary GrandPré with an average score rating of 4.287096774193548.
+ The average number of text reviews among users who rated more than 50 books is 24.333333333333332